# Part 2: Fine Tune a Bert Based Model for sentiment classification

In this section, we will fine-tune distilBert to classify the sentiment of yelp reviews

<a href="https://colab.research.google.com/github/jkchandalia/nlp/blob/main/Poetry_w_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# %%capture
# !pip install transformers
# !pip install datasets
# !pip install evaluate
# !pip install torch

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import numpy as np
import evaluate
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

/Users/d/repos/nlp/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 1: Load, inspect and down-sample our dataset

In [4]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (/Users/d/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 178.72it/s]


In [5]:
dataset.shape, dataset.column_names

({'train': (650000, 2), 'test': (50000, 2)},
 {'train': ['label', 'text'], 'test': ['label', 'text']})

In [6]:
dataset['train'][100]["text"]

'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I\'ve worked at more t

In [7]:
dataset['train']['text'][:10]

["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patient

First we'll train on a small subset of the data to iterate quickly and make sure our code works

In [8]:
# create a new dataset with 100 training samples and 100 test samples
# stratify by column: ensures that the train and test sets have the same proportion of each class as the full dataset
dataset_small = dataset["train"].train_test_split(train_size=100, test_size=100, seed=42, stratify_by_column="label")

In [9]:
dataset_small.shape

{'train': (100, 2), 'test': (100, 2)}

Alternatively: we can define train and test size as a % of the full dataset

```
dataset_small = dataset["train"].train_test_split(train_size=.05, test_size=0.05, seed=42, stratify_by_column="label")
```
In this case, 5% of the original training dataset to our new train set, 5% of the training dataset to our new test set

# Step 2: Tokenize our training and test datasets

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # uncased means lowercase
# alternative tokenizers: distilbert-base-cased (cased means case sensitive), bert-base-uncased, bert-base-cased, roberta-base

help(tokenizer)

Help on DistilBertTokenizerFast in module transformers.models.distilbert.tokenization_distilbert_fast object:

class DistilBertTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  DistilBertTokenizerFast(vocab_file=None, tokenizer_file=None, do_lower_case=True, unk_token='[UNK]', sep_token='[SEP]', pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', tokenize_chinese_chars=True, strip_accents=None, **kwargs)
 |  
 |  Construct a "fast" DistilBERT tokenizer (backed by HuggingFace's *tokenizers* library). Based on WordPiece.
 |  
 |  This tokenizer inherits from [`PreTrainedTokenizerFast`] which contains most of the main methods. Users should
 |  refer to this superclass for more information regarding those methods.
 |  
 |  Args:
 |      vocab_file (`str`):
 |          File containing the vocabulary.
 |      do_lower_case (`bool`, *optional*, defaults to `True`):
 |          Whether or not to lowercase the input when tokenizing.
 |      unk_token (`str`,

In [12]:
def tokenize_function(examples):
    """this function takes in a batch of training (or test) examples and for each, will tokenize the text and truncate or pad it to the max length of 512 tokens. 

    Args:
        examples (list): List of 

    Returns:
        List (int): List of [input IDs] with the appropriate special tokens. The input ids are often the only required parameters to be passed to the model as input. They are token indices, numerical representations of tokens building the sequences that will be used as input by the model. (https://huggingface.co/transformers/v3.2.0/glossary.html#:~:text=The%20input%20ids%20are%20often,as%20input%20by%20the%20model.&text=The%20tokenizer%20takes%20care%20of,available%20in%20the%20tokenizer%20vocabulary.)
    """
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Discussion Question: given the description of the tokenize_function() above, before we proceed, can you think of any potential problems with this approach? How would you go about evaluating if this approach is appropriate for our use-case? (side exercise: ex2_inspect_dataset.ipynb)

In [13]:
# dataset.map applies the tokenize function to all the examples in the dataset. (batched=True means that the function is applied to the examples in batches) Batched=True is faster than batched=False but it requires more memory. It is recommended to use batched=True if you have a GPU and batched=False if you don't have a GPU.
tokenized_datasets = dataset_small.map(tokenize_function, batched=True)

In [19]:
tokenized_datasets

NameError: name 'tokenized_datasets' is not defined

In [52]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [53]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/mo

In [54]:


training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [55]:


metric = evaluate.load("accuracy")

In [56]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [57]:


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [59]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 892
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 336
  Number of trainable parameters = 66957317


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.417676,0.855769
2,No log,0.366288,0.875000
3,No log,0.359098,0.913462


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 104
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 104
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: id, verse_text. If id, verse_text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this m

TrainOutput(global_step=336, training_loss=0.5075324376424154, metrics={'train_runtime': 137.009, 'train_samples_per_second': 19.532, 'train_steps_per_second': 2.452, 'total_flos': 354501723893760.0, 'train_loss': 0.5075324376424154, 'epoch': 3.0})

In [60]:
model.to('cpu')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [85]:

inputs = tokenizer("There once was a girl with a mind so dark, Her thoughts were gloomy, like a night without stars.She saw the world through a veil of pain, And her heart felt heavy, like a ball and chain.", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()


In [86]:
predicted_class_id

0